In [1]:
import pandas as pd
from ggplot import *
import numpy as np
%matplotlib inline

# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

In [11]:
df = pd.read_csv('world_war.csv')

### Quick glance at the data

In [12]:
df.head(10)

,Name,Year,Location,Assaulter,Defender,Assaulter_1,Assaulter_2,Defender_1,As_strength_men,Def_strength_men,As_strength_guns,Def_strength_guns,Assaulter_losses,Defender_losses,Outcome
0,Battle of Mons,1914.0,"Mons, Belgium",German Empire,British Empire,NaN,NaN,NaN,"160,000","80,000",600,300,"5,000","1,638",win
1,Battle of Liege,1914.0,"Liege, Belgium",German Empire,Belgium,NaN,NaN,NaN,"59,800","36,000",400,280,"25,000","15,000",win
2,Battle of the Frontiers,1914.0,"Lorraine, the Ardennes and Belgium",Belgium,German Empire,France,British Empire,NaN,"1,504,000","1,600,000","4,640","5,000","165,000","264,680",loss
3,Togoland Campaign,1914.0,German Togoland,British Empire,German Empire,France,NaN,NaN,"1,100",693,NaN,NaN,137,41,win
4,Battle of Cer,1914.0,Cer Mountain,Austria-Hungary,Serbia,NaN,NaN,NaN,"200,000","180,000",NaN,NaN,"18,500","4,785",loss
5,Battle of Tannenberg,1914.0,"Allenstein, East Prussia",Russian Empire,German Empire,NaN,NaN,NaN,"230,000","150,000",NaN,NaN,"170,000","13,873",loss
6,First Battle of the Marne,1914.0,"Marne River, Paris",France,German Empire,British Empire,NaN,NaN,"1,071,000","1,485,000",NaN,NaN,"263,000","256,000",win
7,Battle of Drina,1914.0,Drina River,Austria-Hungary,Serbia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,loss
8,First Battle of Ypres,1914.0,"Ypres, Belgium",France,German Empire,Belgium,British Empire,NaN,"4,400,000","5,400,000",NaN,NaN,"58,155","46,765",indecisive
9,Siege of Tsingtao,1914.0,"Tsingtau, Germany",British Empire,German Empire,Empire of Japan,NaN,Austria-Hungary,"24,500","3,750",142,NaN,"3,000",700,win


In [13]:
#Create list of Assaulters
assaulter_list = []

for elem in df['Assaulter'].unique():
    if type(elem) == str:
        assaulter_list.append(elem)
    else:
        continue

#Create list of Defenders
defender_list = []

for elem in df['Defender'].unique():
    if type(elem) == str:
        defender_list.append(elem)
    else:
        continue

In [14]:
#Create a list of participants 
participants = []

participants = set(assaulter_list + defender_list)

#How many times has been successfully carried out the attack
wins_at = []

for elem in participants:
    score = 0
    for row in df['Assaulter'][df['Outcome'] == 'win']:
        if elem in row:
            score +=1
        else:
            continue
    wins_at.append(score)

data = {'Assaulter': list(participants),
        'Win': wins_at}
attack_win = pd.DataFrame(data)
wins_at

[1, 1, 2, 2, 0, 6, 6, 1, 0, 0, 0]

In [15]:
#How many times has been unsuccessfully carried out the attack
loss_at = []

for elem in participants:
    score = 0
    for row in df['Assaulter'][df['Outcome'] == 'loss']:
        if elem in row:
            score +=1
        else:
            continue
    loss_at.append(score)

data = {'Assaulter': list(participants),
        'Loss': loss_at}
attack_loss = pd.DataFrame(data)

In [16]:
#How many times has been successfully carried out protection
wins_pr = []

for elem in participants:
    score = 0
    for row in df['Defender'][df['Outcome'] == 'loss']:
        if elem in row:
            score +=1
        else:
            continue
    wins_pr.append(score)

data = {'Defender': list(participants),
        'Win': wins_pr}
protection_win = pd.DataFrame(data)

In [17]:
#How many times has been unsuccessfully carried out protection
loss_pr = []

for elem in participants:
    score = 0
    for row in df['Defender'][df['Outcome'] == 'win']:
        if elem in row:
            score +=1
        else:
            continue
    loss_pr.append(score)

data = {'Defender': list(participants),
        'Loss': loss_pr}
protection_loss = pd.DataFrame(data)


In [18]:
#Dictionary
data = {'attack_win': wins_at,
        'attack_loss': loss_at,
        'protection_win': wins_pr,
        'protection_loss': loss_pr, }

# Create a dataframe from the dictionary, indexed by a participant's name
battle_record = pd.DataFrame(data,index=participants)
battle_record

,attack_loss,attack_win,protection_loss,protection_win
Australia,0,1,0,0
Kingdom of Italy,2,1,0,0
France,2,2,0,0
Bulgaria,0,2,0,0
Russian Empire,1,0,2,0
German Empire,0,6,6,7
British Empire,7,6,3,0
Serbia,0,1,2,2
Austria-Hungary,2,0,2,2
Ottoman Empire,0,0,3,4


In [19]:
# Count the total attacks
battle_record['total_attacks'] = battle_record['attack_win'] + battle_record['attack_loss']

# Count the total defends
battle_record['total_defends'] = battle_record['protection_win'] + battle_record['protection_loss']

# Count the total wins
battle_record['total_wins'] = battle_record['attack_win'] + battle_record['protection_win']

# Count the total losses
battle_record['total_loss'] = battle_record['attack_loss'] + battle_record['protection_loss']

# Create a total number of battles
battle_record['total_battles'] = battle_record['total_attacks'] + battle_record['total_defends']

# Create a win percentage score (total wins / total battles)
battle_record['win_percentage'] = battle_record['total_wins'] / battle_record['total_battles'] * 100

# Create a composite score (total number of wins minus total number of losses)
battle_record['composite_record'] = battle_record['total_wins'] - battle_record['total_loss']

battle_record

,attack_loss,attack_win,protection_loss,protection_win,total_attacks,total_defends,total_wins,total_loss,total_battles,win_percentage,composite_record
Australia,0,1,0,0,1,0,1,0,1,100.000000,1
Kingdom of Italy,2,1,0,0,3,0,1,2,3,33.333333,-1
France,2,2,0,0,4,0,2,2,4,50.000000,0
Bulgaria,0,2,0,0,2,0,2,0,2,100.000000,2
Russian Empire,1,0,2,0,1,2,0,3,3,0.000000,-3
German Empire,0,6,6,7,6,13,13,6,19,68.421053,7
British Empire,7,6,3,0,13,3,6,10,16,37.500000,-4
Serbia,0,1,2,2,1,4,3,2,5,60.000000,1
Austria-Hungary,2,0,2,2,2,4,2,4,6,33.333333,-2
Ottoman Empire,0,0,3,4,0,7,4,3,7,57.142857,1


### Analysis

In [21]:
#Top 10 most active assaulters
battle_record.sort_values('total_attacks', ascending=False).head(10)['total_attacks']

British Empire      13
German Empire        6
France               4
Kingdom of Italy     3
Bulgaria             2
Austria-Hungary      2
Australia            1
Russian Empire       1
Serbia               1
Belgium              1
Name: total_attacks, dtype: int64

In [22]:
#Top 10 most active defenders
battle_record.sort_values('total_defends', ascending=False).head(10)['total_defends']

German Empire       13
Ottoman Empire       7
Serbia               4
Austria-Hungary      4
British Empire       3
Russian Empire       2
Belgium              1
Australia            0
Kingdom of Italy     0
France               0
Name: total_defends, dtype: int64

In [23]:
#Top 10 most victorious participants
battle_record.sort_values('total_wins', ascending=False).head(10)['total_wins']

German Empire       13
British Empire       6
Ottoman Empire       4
Serbia               3
France               2
Bulgaria             2
Austria-Hungary      2
Australia            1
Kingdom of Italy     1
Russian Empire       0
Name: total_wins, dtype: int64

In [24]:
#Top 10 most losing participants
battle_record.sort_values('total_loss', ascending=False).head(10)['total_loss']

British Empire      10
German Empire        6
Austria-Hungary      4
Russian Empire       3
Ottoman Empire       3
Kingdom of Italy     2
France               2
Serbia               2
Belgium              2
Australia            0
Name: total_loss, dtype: int64

In [25]:
#Participants with the best win percentage
battle_record.sort_values('win_percentage', ascending=False).head(75)['win_percentage'][battle_record['win_percentage'] != float('Inf')]

Australia           100.000000
Bulgaria            100.000000
German Empire        68.421053
Serbia               60.000000
Ottoman Empire       57.142857
France               50.000000
British Empire       37.500000
Kingdom of Italy     33.333333
Austria-Hungary      33.333333
Russian Empire        0.000000
Belgium               0.000000
Name: win_percentage, dtype: float64